In [3]:
import tarfile
import urllib
from os.path import isfile, isdir
from tqdm import tqdm_notebook as tqdm

import os
#import cv2
import h5py
import math
import numpy as np
from PIL import Image
from scipy import misc
import random as rand
from operator import add
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches
import numpy as np
import tarfile
import urllib
from IPython.display import display, Image

import scipy.io as sio
import sys
import pandas as pd
%matplotlib inline
from sklearn.model_selection import train_test_split
import glob
import csv

import re
#import nltk
#from nltk.corpus import stopwords
#import nltk.data
from sklearn.utils import shuffle
import zipfile
import pickle

# Load data

In [2]:
train_q1 = np.load('data/train_q1.npy')
train_q2 = np.load('data/train_q2.npy')
train_y = np.load('data/train_y.npy')
cv_q1 = np.load('data/cv_q1.npy')
cv_q2 = np.load('data/cv_q2.npy')
cv_y = np.load('data/cv_y.npy')
test_q1 = np.load('data/test_q1.npy')
test_q2 = np.load('data/test_q2.npy')

In [3]:
print(len(train_q1), len(train_q2), len(train_y), len(cv_q1), len(cv_q2), len(cv_y), len(test_q1), len(test_q2))

323432 323432 323432 80858 80858 80858 2345796 2345796


# Load GloVe

In [4]:
with open("data/glove", 'rb') as f:
    glove = pickle.load(f)

In [7]:
glove('man')

TypeError: 'tuple' object is not callable

In [8]:
from sklearn.preprocessing import normalize
glove_words = glove[0]
glove_vectors = normalize(glove[1])
#glove_vectors = glove[1]

In [19]:
"21" in glove_words

True

In [7]:
del glove

# Transform question to vector

In [8]:
def question_to_vectors_glove(question, length = 20):
    vectors = np.zeros((length,300),dtype = "float32")    
    number_of_words = 0
    
    for idx, word in enumerate(question):        
        if word in glove_words:            
            word_idx = glove_words[word]
            vector = glove_vectors[word_idx]
            vectors[number_of_words] = vector
            number_of_words += 1
            
        if number_of_words == length:
            break          
        
    return vectors

In [9]:
#testing
x = question_to_vectors_glove(["i", "love", "you", "i", "i", "i", "i", "i", "i", "love", "you", "i", "i", "i", "i", "i", "i", "love", "you", "i", "i", "i", "i", "i"
                          "i", "love", "you", "i", "i", "i", "i", "i", "i", "love", "you", "i", "i", "i", "i", "i", "i", "love", "you", "i", "i", "i", "i", "i"])

In [10]:
print(x)
print(type(x), x.shape)

[[-0.00565063 -0.02400844 -0.01898049 ...,  0.01309268  0.01387201
   0.01156845]
 [-0.02002873 -0.00091331 -0.03347111 ..., -0.00618264  0.01985431
   0.0565495 ]
 [-0.00592356 -0.00135805 -0.02587986 ..., -0.05422771  0.01454125
  -0.01574125]
 ..., 
 [-0.02002873 -0.00091331 -0.03347111 ..., -0.00618264  0.01985431
   0.0565495 ]
 [-0.00592356 -0.00135805 -0.02587986 ..., -0.05422771  0.01454125
  -0.01574125]
 [-0.00565063 -0.02400844 -0.01898049 ...,  0.01309268  0.01387201
   0.01156845]]
<class 'numpy.ndarray'> (20, 300)


In [11]:
#testing
q1 = train_q1[1000]
print(q1)
x1 = question_to_vectors_glove(q1)
print(x1)
print(x1.shape)

['what', 'is', 'the', 'formula', 'for', 'finding', 'the', 'least', 'count', 'of', 'a', 'vernier', 'caliper']
[[ 0.01940092 -0.0002964  -0.01272782 ...,  0.00344119  0.0650295
   0.01588619]
 [ 0.01479442  0.04479221 -0.00714847 ..., -0.0483545   0.03737552
  -0.02340546]
 [-0.02676889 -0.01918193 -0.00225168 ..., -0.06945422  0.02723264
  -0.00121213]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
(20, 300)


# Generate data_batch

In [12]:
def data_generator_glove(questions1, questions2, y, batch_size=1024):
    
    sample_size = len(y)
    batch_slices = [slice(i, i + batch_size) for i in range(0, sample_size, batch_size)]
    
    while True:
        for batch in batch_slices:
            batch_new_questions1 = []
            batch_new_questions2 = []
            batch_y = y[batch]

            for question1 in questions1[batch]:
                new_question1 = question_to_vectors_glove(question1)
                batch_new_questions1.append(new_question1)
            
            for question2 in questions2[batch]:
                new_question2 = question_to_vectors_glove(question2)
                batch_new_questions2.append(new_question2)

            yield [np.array(batch_new_questions1), np.array(batch_new_questions2)], np.array(batch_y)

# Train model

In [13]:
from keras import callbacks
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf
import keras
from keras import layers

Using TensorFlow backend.


In [14]:
def tower(output_dim, num_of_layers, activation, dropout, input_dim=(20, 300)):
    x = layers.Input(input_dim)
    
    if num_of_layers == 1:

        rep = layers.recurrent.LSTM(output_dim,  
                                    input_shape=input_dim,  
                                    return_sequences=False, 
                                    activation=activation,
                                    dropout=dropout)(x)

        rep = layers.Dropout(dropout)(rep)
    
    if num_of_layers == 2:
    
        rep = layers.recurrent.LSTM(output_dim,  
                                    input_shape=input_dim,  
                                    return_sequences=True, 
                                    activation=activation,
                                    dropout=dropout)(x)

        rep = layers.Dropout(dropout)(rep)
        
        rep = layers.recurrent.LSTM(output_dim)(rep)

        
    return x, rep

In [30]:
def merge_two_towers(left, right, method):
    if method == "concat":
        return layers.concatenate([left, right])
    elif method == "multiply":
        return layers.multiply([left, right])
    elif method == "ave":
        return layers.average([left, right])
    elif method == "dif":
        return layers.merge([left, right], mode=lambda x: x[0] - x[1], output_shape=lambda x: x[0])
    elif method == "cos":
        return layers.merge([left, right], mode="cos")

In [33]:
def train_model(tower_dim, tower_layer, tower_act, tower_dropout,
               merge_method,
               dense_dim, dense_act,
               optimizer):
    
    in1, left = tower(tower_dim, tower_layer, tower_act, tower_dropout)
    in2, right = tower(tower_dim, tower_layer, tower_act, tower_dropout)

    rep = merge_two_towers(left, right, merge_method)
    
    rep = layers.Dropout(0.5)(rep)
    
    output = layers.Dense(dense_dim, activation=dense_act)(rep)

    model = keras.models.Model(inputs=[in1, in2], outputs=[output])
    
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    model.fit_generator(data_generator_glove(train_q1, train_q2, train_y),
                        steps_per_epoch=len(train_y)//1024, epochs=10, 
                        validation_data=data_generator_glove(cv_q1, cv_q2, cv_y),
                        validation_steps = len(cv_y)//1024)
    return model

In [34]:
model1 =  train_model(tower_dim=100, tower_layer=1, tower_act="softsign", tower_dropout=0.5,
               merge_method="concat",
               dense_dim=2, dense_act="softmax",
               optimizer="adam")

Epoch 1/10
315/315 [==============================] - 34s - loss: 0.6053 - acc: 0.6702 - val_loss: 0.5752 - val_acc: 0.7045
Epoch 2/10
315/315 [==============================] - 34s - loss: 0.5840 - acc: 0.6920 - val_loss: 0.5654 - val_acc: 0.7092
Epoch 3/10
170/315 [===============>..............] - ETA: 12s - loss: 0.5777 - acc: 0.6982

KeyboardInterrupt: 